# 01 Select sites to process

In [ ]:
import os, sys, datetime, pandas as pd
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # emplacement des modules RenPri

from issep import sarsar_admin
from os.path import join
from lecture_ini import config

In [ ]:
def sites_to_process(dteDebut, dteFin, summerdteChgEnd, dteExe):
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass

    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env at the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None
    cur2 = None

    try:
        import psycopg2.extras #permet de communiquer avec pgsql
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

        # Liste les sar_id_segment pour lesquels j'ai des observations dans l'intervalle de dates
        strSQL = '''SELECT DISTINCT s.sar_id_segment 
                    FROM sar_index_stats s, sar_def d WHERE s.sar_id_segment = d.id_segment 
                    AND d.is_active = 't' 
                    AND s.index_name NOT IN ('BI2','VV','SBI','NDVI','BI2_part1','BI','BAI','VH') 
                    AND substring(s.index_name,1,2) != 'VV' AND s.acq_date BETWEEN '{0}' AND '{1}' 
                    ORDER BY s.sar_id_segment;'''.format(dteDebut, dteFin)
        cur.execute(strSQL)
        sites = [item[0] for item in cur.fetchall()]

        return sites
            
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        return []
    
    finally:
        if cur is not None:
            cur.close()

    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()